In [6]:
#convert multistring shapefile of digitized boundaries to binary mask
import os
import numpy as np
from osgeo import gdal, ogr

# Function to create a binary mask from a shapefile
def create_binary_mask(shapefile_path, raster_path, output_mask_path):
    # Open shapefile
    shape_ds = ogr.Open(shapefile_path)
    shape_layer = shape_ds.GetLayer()

    # Get extent of shapefile
    x_min, x_max, y_min, y_max = shape_layer.GetExtent()

    # Open raster dataset to get size
    raster_ds = gdal.Open(raster_path)
    raster_width = raster_ds.RasterXSize
    raster_height = raster_ds.RasterYSize

    # Create a new raster to store the mask
    mask_ds = gdal.GetDriverByName('GTiff').Create(output_mask_path, raster_width, raster_height, 1, gdal.GDT_Byte)
    mask_ds.SetProjection(raster_ds.GetProjection())
    mask_ds.SetGeoTransform(raster_ds.GetGeoTransform())

    # Rasterize shapefile onto mask
    gdal.RasterizeLayer(mask_ds, [1], shape_layer, burn_values=[1])

    # Close datasets
    del mask_ds, raster_ds, shape_ds

# Paths
shapefile_path = '/home/jovyan/Tushar/Tile_2.shp'
raster_path = "/home/jovyan/Tushar/FBD_Task/High_Resolution_image_for_training/jag_t2.tif"
output_mask_path = "/home/jovyan/Tushar/mask.tif"

# Create binary mask
create_binary_mask(shapefile_path, raster_path, output_mask_path)

print("Binary mask generated and saved successfully.")


Binary mask generated and saved successfully.


In [ ]:
#generating patches of image and mask for training
import cv2
import os

def generate_patches(image_path, mask_path, patch_size, output_dir_images, output_dir_masks):
    # Create output directories if they don't exist
    os.makedirs(output_dir_images, exist_ok=True)
    os.makedirs(output_dir_masks, exist_ok=True)
    
    # Load images
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    
    # Get image dimensions
    height, width, _ = image.shape
    
    # Calculate number of patches
    num_patches_height = height // patch_size
    num_patches_width = width // patch_size
    
    # Generate patches
    for i in range(num_patches_height):
        for j in range(num_patches_width):
            y_start = i * patch_size
            y_end = (i + 1) * patch_size
            x_start = j * patch_size
            x_end = (j + 1) * patch_size
            
            # Extract patches from image and mask
            image_patch = image[y_start:y_end, x_start:x_end]
            mask_patch = mask[y_start:y_end, x_start:x_end]
            
            # Save patches
            cv2.imwrite(os.path.join(output_dir_images, f"image_patch_{i}_{j}.tif"), image_patch)
            cv2.imwrite(os.path.join(output_dir_masks, f"mask_patch_{i}_{j}.tif"), mask_patch)

# Paths and parameters
image_path = "/home/jovyan/Tushar/FBD_Task/High_Resolution_image_for_training/jag_t2.tif"
mask_path = "/home/jovyan/Tushar/mask.tif"
patch_size = 256  # Adjust patch size as needed
output_dir_images = "/home/jovyan/Tushar/FBD_Task/image_patches"
output_dir_masks = "/home/jovyan/Tushar/FBD_Task/mask_patches"

# Generate patches
generate_patches(image_path, mask_path, patch_size, output_dir_images, output_dir_masks)
